### 自动求导例子

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
# 在我们计算 𝑦 关于 𝐱 的梯度之前，我们需要一个地方来存储梯度
x.requires_grad_(True)  # 等价于 `x = torch.arange(4.0, requires_grad=True)`
x.grad  # 默认值是None

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
# x是一个长度为4的向量，计算x和x的内积，得到了我们赋值给y的标量输出。接下来，我们可以通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 非标量变量的反向传播

In [10]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数关于`self`的梯度。在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 分离计算

In [11]:
# 在这里，我们可以分离y来返回一个新变量u，该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息。换句话说，梯度不会向后流经u到x。
# 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理，而不是z=x*x*x关于x的偏导数。
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [13]:
# 由于记录了y的计算结果，我们可以随后在y上调用反向传播，得到y=x*x关于的x的导数，这里是2*x
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### Python控制流的梯度计算

In [17]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [18]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [33]:
a.grad == d / a

tensor(True)

In [34]:
d,a

(tensor(1486.4039, grad_fn=<MulBackward0>), tensor(1.4516, requires_grad=True))

In [35]:
a.grad

tensor(1024.)